In [1]:

import chart_studio.plotly as py
import chart_studio
username = '' # your username
api_key = '' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

from scipy.io import loadmat

def read_HSI():
  X = loadmat('Datasets\IndianPines\Indian_pines_corrected.mat')['indian_pines_corrected']
  y = loadmat('Datasets\IndianPines\Indian_pines_gt.mat')['indian_pines_gt']
  print(f"X shape: {X.shape}\ny shape: {y.shape}")
  return X, y

X, y = read_HSI()

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
'''
sns.axes_style('whitegrid')
fig = plt.figure(figsize = (12, 6))

for i in range(1, 1+6):
    fig.add_subplot(2,3, i)
    q = np.random.randint(X.shape[2])
    plt.imshow(X[:,:,q], cmap='jet')
    plt.axis('off')
    plt.title(f'band - {q}')
'''
import plotly.express as px
import pandas as pd
import numpy as np

def extract_pixels(X, y):
  q = X.reshape(-1, X.shape[2])
  df = pd.DataFrame(data = q)
  df = pd.concat([df, pd.DataFrame(data = y.ravel())], axis=1)
  df.columns= [f'band{i}' for i in range(1, 1+X.shape[2])]+['class']
  return df
  
df = extract_pixels(X, y)

df.head()

df.info()

df.iloc[:, :-1].describe()

sample_size = 1
sample = df.groupby('class').apply(lambda x: x.sample(sample_size))
sample.head()
sample = sample.T
sample.columns = [str(i) for i in range(17)]

fig = px.line(sample,
          x=sample.index,
          y=str(0))

for i in range(1,16):
  fig.add_scatter(x=sample.index, y=sample[str(i)])
fig.show()



In [ ]:
from preprocess import *
from utils import *
from pandas import DataFrame
dataset = "IndianPines"
preprocesses = ['pca', 'ica','lda']
hsi_img, gt, label_values, ignored_labels = load_dataset(dataset)
class_labels = {'1': '1'	,
    '2'	:'2'	,
    '3'	:'3'	,
    '4'	:'4'	,
    '5'	:'5'	,
    '6'	: '6'	,
    '7'	:'7'	,
    '8'	:'8'	,
    '9'	:'9'    ,
    '10': '10'	,
    '11': '11'	,
    '12': '12'	,
    '13': '13'	,
    '14': '14'	,
    '15': '15'	,
    '16': '16'	
    }
for i in range(len(preprocesses)):
    img = preprocess(hsi_img, gt, preprocess_name=preprocesses[i], n_bands=3)
    img_merge = np.concatenate((img, gt[:,:,None]), axis=2)
    m, n, p = img_merge.shape
    img_data = np.reshape(img_merge, (m * n, p))
    img_frame = DataFrame(img_data,index=None,columns = ["PC-1","PC-2","PC-3",'class'])
    img_frame['class'] = img_frame['class'].map(lambda x: int(x))
    sample_size = 20
    sample = img_frame.groupby('class').apply(lambda x: x.sample(sample_size))
    sample = sample[sample['class'] != 0]
    sample['class'].value_counts()
    sample['label'] =  sample['class'].apply(lambda x : class_labels[str(x)])
    sample['label'].value_counts()
    scatter_3d = px.scatter_3d(sample, x="PC-1", y="PC-2", z="PC-3", size="class", color = "label",hover_name="label",
                    symbol="label")#, color_discrete_map = {"Joly": "blue", "Bergeron": "green", "Coderre":"red"})
    scatter_3d.show()